# MRIQC

In [36]:
import sys
from subprocess import check_output, call
import pandas as pd
import numpy as np
#from utils import exclude_subjects, select_subjects
import os
import shutil

data_dict_fname = os.getcwd() + '/data_dict.npz'

task = 'ARC'
fsfast_path = os.getcwd() + '/data/derivatives/fmri_first_levels'
fmriprep_path = os.getcwd() + '/data/derivatives/fmriprep'
user_id = '4190106' #type id in a terminal user is your user group is the last one (iliad)
group_id = '1047'
data_dir = '/space/lilli/3/users/DARPA-TRANSFER/mri'
fs_license = '/autofs/space/karima_001/users/alex/software/license.txt'

if not os.path.exists(fsfast_path):
    os.makedirs(fsfast_path)
if not os.path.exists(fmriprep_path):
    os.makedirs(fmriprep_path)

# absolute paths needed for docker
root = '/autofs/space/karima_004/users/learning_mri/'
data_path = root + 'data'
mriqc_path = '%s/derivatives/mri/mriqc' % data_path

# extract mri subjects 
info = pd.read_csv(root + 'mri_subjects.csv')
info = info.loc[info[task.title()], 'Subject'].reset_index()
subjects = list(info.Subject)
with open('%s/subjects' % fsfast_path, 'w') as fid:
    for s in subjects:
        fid.write(s + '\n')
        
np.savez_compressed(data_dict_fname,task=task,fsfast_path=fsfast_path,fmriprep_path=fmriprep_path,
                    user_id=user_id,group_id=group_id,data_dir=data_dir,root=root,data_path=data_path,
                    mriqc_path=mriqc_path,subjects=subjects,fs_license=fs_license)

In [10]:
print(check_output(['docker',
                    'run', 
                    'poldracklab/mriqc:latest',
                    '--version']))

mriqc v0.10.1



## Generate Subject Reports

In [12]:
data_dict = np.load(os.getcwd() + '/data_dict.npz')
user_id, group_id, mriqc_path = data_dict['user_id'].item(),data_dict['group_id'].item(),data_dict['mriqc_path'].item()
docker_command = ['docker',
                  'run',
                  '--rm',
                  '-u', user_id + ':' + group_id,
                  '-v', '%s:/data:ro' % data_path,
                  '-v', '%s:/out' % mriqc_path,
                  '-v', '%s:/work' % mriqc_path,
                  'poldracklab/mriqc:latest',
                  '/data', '/out',
                  'participant',
                  '-w', '/work',
                  '--no-sub',
                  '--verbose-reports',
                  '--write-graph',
                  '--ica',
                  '--n_procs=10',
                  '--fft-spikes-detector',
                  '--fd_thres', '0.9']

folders = ['anatMRIQCT1w','derivatives','funcMRIQC','logs','reports','workflow_enumerator']
for folder in folders:
    if not os.path.isdir(mriqc_path + '/' + folder):
        os.makedirs(mriqc_path + '/' + folder)
        
print(' '.join(docker_command))
call(docker_command)

4190106
docker run --rm -u 4190106:1047 -v /autofs/space/karima_004/users/learning_mri/data:/data:ro -v /autofs/space/karima_004/users/learning_mri/data/derivatives/mri/mriqc:/out -v /autofs/space/karima_004/users/learning_mri/data/derivatives/mri/mriqc:/work poldracklab/mriqc:latest /data /out participant -w /work --no-sub --verbose-reports --write-graph --ica --n_procs=10 --fft-spikes-detector --fd_thres 0.9


KeyboardInterrupt: 

## Generate Group Reports

In [28]:
data_dict = np.load(os.getcwd() + '/data_dict.npz')
user_id, group_id, mriqc_path = data_dict['user_id'].item(),data_dict['group_id'].item(),data_dict['mriqc_path'].item()
docker_command = ['docker',
                  'run',
                  '--rm',
                  '-u', user_id + ':' + group_id,
                  '-v', '%s:/data:ro' % data_path,
                  '-v', '%s:/out' % mriqc_path,
                  '-v', '%s:/work' % mriqc_path,
                  'poldracklab/mriqc:latest',
                  '/data', '/out',
                  'group',
                  '-w', '/work']
print(' '.join(docker_command))
call(docker_command)

docker run --rm -u 4190106:1047 -v /autofs/space/karima_004/users/learning_mri/data:/data:ro -v /autofs/space/karima_004/users/learning_mri/data/derivatives/mri/mriqc:/out -v /autofs/space/karima_004/users/learning_mri/data/derivatives/mri/mriqc:/work poldracklab/mriqc:latest /data /out group -w /work


KeyboardInterrupt: 

## Observations & Subject Exclusions

We use the group reports for the T1 and BOLD scans to determine exclusions:

- <a href="./data/derivatives/mri/mriqc/reports/T1w_group.html"> T1 Group Report</a>
- <a href="./data/derivatives/mri/mriqc/reports/bold_group.html"> BOLD Group Report</a>

T1 Observations:
* pre-scan normalize was off for the T1 on the bay 4 prisma
* The newer prisma has much better noise quality than the older trio scanners as measured by EFC (measure of ghosting/blurring) & FBER (relative energy within brain relative to background). Hard to actually see this however due to lack of pre-scan normalize for the prisma scans. This difference also appears whenever anything is normalized by background noise (such as SNRD).
* The CJV & CNR measures appeared to be inversely related with heavy tails. The tails seemed ok and not grounds for exclusion. It seems it may have been sensitive to the amount of gyral folding and hence the discriminability of grey and white matter?
* A few other outliers, but most related to wrap around, ghosting, etc. that did not affect the brain itself (apart from exclusions below).

BOLD Observations:
* A few with fairly bad motion. Leaving in to see if scrubbing/correction can help.
* sub-hc045 has weird frontal dropout (not present in T1). Leaving in to see if b0 correction helps. Outlier on FWHM y.
* sub-hc041 lots of motion may be able to censor

T1 Observations:
* sub-hc034 spiking, likely poor recon

Subject Exclusions:
* sub-pp004: Motion
* sub-ep011: Motion
* sub-hc037: Appears to not have had anterior head coil in. Caught by being outlier in FWHM y for T1 and bold.
* sub-hc020: Extremely bad motion. Caught as outlier on AOR and Average FD.
* sub-hc047: Really bad motion and really bad distortion/dropout in the frontal regions. Caught as outlier on Average FD and FWHM y.

In [ ]:
import pandas as pd
import os

subjects = ['hc020', 'pp004', 'hc037', 'hc047','ep011']

f = os.getcwd() + '/mri_subjects.csv'
sub_info = pd.read_csv(f)
if 'exclude' not in sub_info:
    sub_info['exclude'] = False
sub_info.loc[sub_info.Subject.isin(subjects),'exclude'] = True
sub_info.to_csv(f, index=False)

# Preprocessing with FMRIPREP

Here we use the Poldrack lab's <a href="https://fmriprep.readthedocs.io/en/1.0.0-rc2/">fmriprep v1.0.0-rc2</a> software package to perform preprocessing of the BOLD data. fmriprep is an awesome tool built off of nipype that combines different preprocessing steps across multiple packages into a single preprocessing workflow. 


The full workflow is detailed <a href="https://fmriprep.readthedocs.io/en/1.0.0-rc2/workflows.html#">here</a>. The workflow does many things including generating freesurfer reconstructions, motion correction, beta0 field correction, slice time correction, and more.

In [13]:
import sys
from subprocess import check_output, call

print(check_output(['docker',
                    'run', 
                    'poldracklab/fmriprep:latest',
                    '--version']))

fmriprep v1.0.6-2



## Run fmriprep


We run the fmriprep command via the wrapper script run_fmriprep.py. This is a python wrapper for generating the docker run command in order to run fmriprep. The command for each subject looks as follows: 

    docker run --rm -v $data_path:/data:ro -v $fmriprep_path:/out -v $fmriprep_path:/work -v $FS_LICENSE:/opt/freesurfer/license.txt -w /work poldracklab/fmriprep:latest /data /out participant --participant_label $subject -t $task -w /work --omp-nthreads 1 --nthreads 1 --output-space fsaverage fsnative T1w template 
    
This command runs the fmriprep command with the bids data folder (./data) as input and it stores its output in the ./data/derivatives folder. We transform the preprocessed functional data into both the native surface (fsnative) and T1 (T1w) space as well as normalized fsaverage and MNI template spaces. The write graph option writes out the computation graph which is pictured below.

Parallelization Note: I explicitly set the number of threads for each fmriprep call to 1. 
Instead, I parallelize across subjects by using the subprocess.Popen command to start a new process for each separate subject. Without parallelization, preprocessing would take ~60 days (1 day per subject), but I was able to make use of 15 cores at a time on my computer which reduced the processing time to ~4 days since I could thus process 15 subjects at a time. Trying to run all subjects at once with fmriprep with the number of threads set greater than 1 did not seem to actually parallelize across subjects and was much slower. Note that the memory usage gets quite intense for this (~30 GB for 15 subjects).



In [37]:
import numpy as np
data_dict = np.load(os.getcwd() + '/data_dict.npz')
task, subjects, fmriprep_path,fs_license,data_path = data_dict['task'].item(),data_dict['subjects'],data_dict['fmriprep_path'].item(),data_dict['fs_license'].item(),data_dict['data_path'].item()
user_id,group_id = data_dict['user_id'].item(),data_dict['group_id'].item()
import sys
from subprocess import check_output, call

docker_command = ['docker', 'run', '--rm',
                  '-u', '%s:%s' %(user_id, group_id),
                  '-v', '%s:/data:ro' % data_path,
                  '-v', '%s:/out' % fmriprep_path,
                  '-v', '%s:/work' % fmriprep_path,
                  '-v', '%s:/opt/freesurfer/license.txt' %(fs_license),
                  '-w', '/work', 'poldracklab/fmriprep:latest',
                  '/data', '/out','participant',
                  '--participant_label','sub-ep011',
                  '-t', task,
                  '-w', '/work',
                  '--omp-nthreads', '1',
                  '--nthreads', '1',
                  '--output-space', 'fsaverage', 'fsnative', 'T1w', 'template'
                  ]

print(' '.join(docker_command))
#call(docker_command)

docker run --rm -u 4190106:1047 -v /autofs/space/karima_004/users/learning_mri/data:/data:ro -v /autofs/space/karima_004/users/learning_mri/data/derivatives/fmriprep:/out -v /autofs/space/karima_004/users/learning_mri/data/derivatives/fmriprep:/work -v /autofs/space/karima_001/users/alex/software/license.txt:/opt/freesurfer/license.txt -w /work poldracklab/fmriprep:latest /data /out participant --participant_label sub-ep011 -t Learning -w /work --omp-nthreads 1 --nthreads 1 --output-space fsaverage fsnative T1w template


## Make Compatible with FSFAST for 1st levels

To do this, we need to transfer data into correct hierarchy (T1 sampled data and mask). Then run preproc-sess with spatial resampling and smoothing and everything else checked no.

### Make FSFAST Directory & Transfer Data

In [38]:
import os
import pandas as pd
import numpy as np
import shutil

data_dict = np.load(os.getcwd() + '/data_dict.npz')
task, subjects, fsfast_path,data_dir = data_dict['task'].item(),data_dict['subjects'],data_dict['fsfast_path'].item(),data_dict['data_dir'].item()
                
for sub in subjects:
    print(sub)
    # find day info
    for i in range(1,4):
        fname = '%s/%s_0%i/cfg.txt' % (data_dir, sub.upper(),i)
        if os.path.isfile(fname):
            cfg = pd.read_csv(fname,sep='\s+')
            if any([task.title() in scan for scan in cfg.ix[:,-1]]):
                task_day = i      
    # make the subject folder hierarchy
    sub_dir = '%s/%s/%s/00%i/masks' % (fsfast_path, sub, task, task_day)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)
    sub_dir = '%s/%s/%s/masks' % (fsfast_path, sub, task)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)
    # add a fsfast subject name indicator
    with open('%s/%s/subjectname' % (fsfast_path, sub), 'w') as fid:
        fid.write(sub)
    # copy over the bold data
    # copied as f and fmcpr because I have to trick fsfast since it doesn't
    # seem to actually support skipping motion correction
    src = '%s/fmriprep/sub-%s/func/sub-%s_task-%s_bold_space-T1w_%s.nii.gz'
    dest1 = '%s/%s/%s/00%i/f.nii.gz' 
    dest2 = '%s/%s/%s/00%i/fmcpr.nii.gz' 
    shutil.copy(src % (fmriprep_path, sub, sub, task.title(), 'preproc'),
                dest1 % (fsfast_path, sub, task, task_day))
    shutil.copy(src % (fmriprep_path, sub, sub, task.title(), 'preproc'),
                dest2 % (fsfast_path, sub, task, task_day))
    # copy over the brainmask
    dest1 = '%s/%s/%s/00%i/masks/brain.nii.gz' 
    dest2 = '%s/%s/%s/masks/brain.nii.gz' 
    shutil.copy(src % (fmriprep_path, sub, sub, task.title(),'brainmask'),
                dest1 % (fsfast_path, sub, task, task_day))
    shutil.copy(src % (fmriprep_path, sub, sub, task.title(),'brainmask'),
                dest2 % (fsfast_path, sub, task))
    
print('Done!')

ep003


/autofs/space/karima_001/users/alex/software/anaconda2.7/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


ep006
ep008
ep009
ep011
hc001
hc003
hc004
hc005
hc006
hc007
hc008
hc009
hc010
hc011
hc012
hc013
hc015
hc016
hc017
hc018
hc019
hc020


IOError: [Errno 2] No such file or directory: '/autofs/space/karima_004/users/learning_mri/data/derivatives/fmriprep/fmriprep/sub-hc020/func/sub-hc020_task-Learning_bold_space-T1w_preproc.nii.gz'

### Spatial Normalization & Smoothing

The two remaining steps beyond fmriprep that we need to do are sample to common spatial spaces (fsaverage for surface and mni305 for subcortical volume) and then spatially smooth the signal. We re-sample to commons spaces so that we can compare across subjects (individual spaces wouldn't line up). We spatially smooth the resulting signals with gaussian kernels (fwhm 4 voxels). This value was determine at Nichol's recommendation from the tfce paper where optimal results were found with a spatial smoothing sigma of 1.5 voxels = 2.335 * 1.5 rounded to 4.

Unfortunately, to play nicely with fsfast's first level analysis pipeline which I need (not so easy to implement the autocorrelation correction), I have to run a few other things. This includes re-registering the functional data to the T1. Since they are already aligned this process should be good and a sanity check below confirms that.

The last output is I use fsfsast's mkbrainmask-sess to compute the global mean value using a temporary brainmask produced by fsfast. This value is used to normalize the images for input to the first level analysis.

In [ ]:
%%bash

task=learning
# Source freesurfer v6
export FREESURFER_HOME=/usr/local/freesurfer/stable6_0_0
. /usr/local/freesurfer/stable6_0_0/SetUpFreeSurfer.sh
export SUBJECTS_DIR=/autofs/space/karima_001/users/alex/$task/data/derivatives/freesurfer

# get into fsfast directory
export FSFAST=./data/derivatives/fmri_first_levels
cd $FSFAST

# make the template run
mktemplate-sess -sf subjects -d . -fsd $task -update

# co-register bold to t1
register-sess -sf subjects -d . -fsd $task -delete-dat -dof 6 -bold -per-run -update -init-coreg

# Run here to compute global mean value (delete tmp brainmasks afterwards)
mkbrainmask-sess -maskstem brainn -fsd $task -sf subjects -d . -per-run -update
rm -r */$task/masks/brainn*
rm -r */$task/001/masks/brainn*

# sample to fsaverage surface
rawfunc2surf-sess -fwhm 4 -sf subjects -d . -fsd $task -trgsubject fsaverage -update -per-run

# sample to mni305 subcortical volume
rawfunc2tal-sess -fwhm 4 -sf subjects -d . -fsd $task -update -subcort-mask -per-run

### Sanity Check Registrations

The bold had already been sampled to the T1 space, but this is just a nice sanity check to make sure nothing went wrong with the registration process here

In [1]:
import os
import pandas as pd
import shutil

task = 'learning'

fsfast_path = os.getcwd() + '/data/derivatives/fmri_first_levels'

info = pd.read_csv(os.getcwd() + '/mri_subjects.csv')
info = info.loc[info[task.title()], 'Subject'].reset_index()
subjects = list(info.Subject)

In [2]:
%%bash
task=learning
# Source freesurfer v6
export FREESURFER_HOME=/usr/local/freesurfer/stable6_0_0
. /usr/local/freesurfer/stable6_0_0/SetUpFreeSurfer.sh
export SUBJECTS_DIR=/autofs/space/karima_001/users/alex/$task/data/derivatives/freesurfer

# get into fsfast directory
export FSFAST=./data/derivatives/fmri_first_levels
cd $FSFAST

tkregister-sess -sf subjects -fsd $task -per-run -bbr-sum

ERROR: cound not find session ep001 ERROR: cound not find session ep001 ERROR: cound not find session ep002 ERROR: cound not find session ep002 ERROR: cound not find session ep004 ERROR: cound not find session ep005 ERROR: cound not find session ep005 ERROR: cound not find session ep007 ERROR: cound not find session ep012 ERROR: cound not find session hc002 ERROR: cound not find session hc002 ERROR: cound not find session hc014 ERROR: cound not find session hc014 ERROR: cound not find session hc024 ERROR: cound not find session hc024 ERROR: cound not find session hc040 ERROR: cound not find session pp013 ERROR: cound not find session pp013 ERROR: cound not find session sp001 ERROR: cound not find session sp001 ERROR: cound not find session sp002 ERROR: cound not find session sp002 ERROR: cound not find session sp003 ERROR: cound not find session sp003 ERROR: cound not find session sp006 ERROR: cound not find session sp006 ERROR: cound not find session sp007 ERROR: cound not find sessio

## Assemble Behavior

In [2]:
import os
import numpy as np
from pandas import read_csv
root_dir = '/autofs/space/karima_004/users/learning_mri/data/'

observed = 0.90    # 90% or more of possible data must be present
accuracy = 0.25    # 25% accuracy or higher

csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('mri-1')])

if not os.path.isdir('fsfast'):
    os.path.mkdirs('fsfast')
with open('fsfast/sessid', 'w') as sessid:
    for f in csv_files:
        ## Get subject name.
        subject = f.split('_')[0]
        ## Load CSV.
        df = read_csv(os.path.join(root_dir,f))
        df = df[df.Condition != 0].reset_index(drop=True)   # Drop rest trials.
        ## Assess quality.
        if (df.ResponseAccuracy != 99).mean() < observed: continue
        else: df = df[df.ResponseAccuracy != 99].reset_index(drop=True)
        if df.ResponseAccuracy.mean() < accuracy: continue
        else: sessid.write('%s\n' %subject)
    
## Save.
sessid.close()

## Construct Regressors

In [5]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.special import gammaln
import nibabel as nib


root_dir = '/autofs/space/karima_001/users/alex/learning/data/'
mri_dir = '/autofs/space/karima_001/users/alex/learning/data/'

task = 'learning'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define useful functions.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def spm_hrf(RT, P=None, fMRI_T=16):
    p = np.array([6, 16, 1, 1, 6, 0, 32], dtype=float)
    if P is not None:
        p[0:len(P)] = P
    _spm_Gpdf = lambda x, h, l: np.exp(h * np.log(l) + (h - 1) * np.log(x) - (l * x) - gammaln(h))
    # modelled hemodynamic response function - {mixture of Gammas}
    dt = RT / float(fMRI_T)
    u = np.arange(0, int(p[6] / dt + 1)) - p[5] / dt
    with np.errstate(divide='ignore'):  # Known division-by-zero
        hrf = _spm_Gpdf(u, p[0] / p[2], dt / p[2]) - _spm_Gpdf(u, p[1] / p[3],
                                                               dt / p[3]) / p[4]
    idx = np.arange(0, int((p[6] / RT) + 1)) * fMRI_T
    hrf = hrf[idx]
    hrf = hrf / np.sum(hrf)
    return hrf

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

subjects = np.loadtxt('fsfast/sessid', dtype='str')

for subject in subjects:
    
    print subject,
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Get params.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    bold_fname = mri_dir + 'sub-%s/func/sub-%s_task-%s_bold.nii' %(subject,subject,task.title())
    img = nib.load(bold_fname) 
    v1,v2,v3,tr = img.header.get_zooms()
    xshape,yshape,zshape,n_acq = img.get_data().shape
    sfreq = float(zshape/tr)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Initialize regressors.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

    ## Setup timing information.
    total_time = n_acq * tr
    times = np.arange(0, total_time+1./sfreq, 1./sfreq)
    n_times = times.shape[0]

    ## Load behavior information.
    df = read_csv(os.path.join(root_dir, 'sub-%s/mri/%s_%s_mri-1' %(subject,subject,task)))
    df = df[df.Condition != 0]            # Drop rest trials.
    df = df[df.ResponseAccuracy != 99]    # Drop missing trials.
    
    ## Define contrasts.
    conditions = np.unique(df.Condition)
    n_conditions = len(conditions)
    
    ## Initialize boxcars.
    neural_signal = np.zeros((n_conditions,n_times))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Generate boxcars.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    for n in range(n_conditions):
        ## Extract onsets/offsets.
        onsets  = df.loc[df.Condition==n+1, 'StimOnset'].as_matrix()
        offsets = df.loc[df.Condition==n+1, 'ResponseOnset'].as_matrix()
        ## Round onsets/offsets.
        onsets  = onsets.round(np.log10(sfreq).astype(int))
        offsets = offsets.round(np.log10(sfreq).astype(int))   
        ## Iteratively generate boxcars.
        for onset, offset in zip(onsets,offsets): 
            mask = (times >= onset) & (times <= offset)
            neural_signal[n,mask] = 1

    ## Perform convolution.
    hrf = spm_hrf(1./sfreq)
    bold_signal = np.apply_along_axis(np.convolve, 1, neural_signal, v=hrf)
    bold_signal = bold_signal[:,:neural_signal.shape[-1]] # Set back to original length.
    
    ## Downsample to start of TR.
    tr_onsets = np.insert( np.cumsum( np.ones(n_acq-1)*tr ), 0, 0 )
    ds = np.array([abs(tr_onsets-t).min() < 0.00001 for t in times])
    if not ds.sum() == n_acq: raise ValueError('Oh noes!')
    bold_signal = bold_signal[:,ds]
    
    ## Normalize regressors. [See Calhoun et al. (2004)]
    ## First we normalize [Intercept, DDB, Valence] such that the sum
    ## of their timeseries squared is equal to 1.
    sums = np.power(bold_signal,2).sum(axis=1)
    bold_signal = (bold_signal.T / np.sqrt(sums)).T
    
    ## Save task regressors.
    for arr, label in zip(bold_signal, conditions):
        fdir = mri_dir + 'sub-%s/%s_001/' %(subject,task)
        if not os.path.isdir(fdir):
            os.makedirs(fdir)
        f = fdir + '%s.par' %(label)
        np.savetxt(f, arr[:,np.newaxis], fmt='%s')
        
    np.savez(fdir + 'params.npz',tr=tr,sfreq=sfreq,n_acq=n_acq,tr_onsets=tr_onsets)
            
print 'Done.'

ep006 ep008 hc001 hc003 hc005 hc006 hc007 hc008 hc009 hc010 hc011 hc012 hc013 hc015 hc016 hc017 hc019 hc020 hc021

/autofs/space/karima_001/users/alex/software/anaconda2.7/lib/python2.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in divide


 hc022 hc025 hc026 hc028 hc029 hc030 hc031 hc032 hc033 hc034 hc035 hc036 hc038 hc042 hc045 hc047 pp002 pp003 pp004 pp005 pp006 pp007 pp008 pp009 pp010 pp012 pp014 pp015 pp016 sp004 sp005 Done.


## Construct Time Point Censors

In [ ]:
import os
import numpy as np
from pandas import read_csv
from scipy.signal import detrend
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

mri_dir = '/autofs/space/karima_001/users/alex/learning/data/'
task = 'learning'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Scrubbing parameters.
thresholds = [0.0, 0.5, 1.0]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Get subjects list.
subjects = np.loadtxt('fsfast/sessid', dtype='str')
with open('fmri/nuisance_info.csv','w') as info:
    info.write('Subject,n_mc,FD=0.0,FD=0.5,FD=1.0\n')
    for subject in subjects:
        info.write('%s,' %subject)
        f = np.load(mri_dir + 'sub-%s/%s_001/params.npz' %(subject,task))
        tr,sfreq,n_acq,tr_onsets = f['tr'].item(),f['sfreq'].item(),f['n_acq'].item(),f['tr_onsets']
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute framewise displacement.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Read motion data.
        mc = os.path.join(mri_dir, subject, 'msit_001', '001', 'fmcpr.mcdat')
        try: mc = np.loadtxt(mc)[:,1:7]
        except IOError: 
            print 'Drop %s.' %subject
            continue

        ## Invert angular displacement.
        fd = mc.copy()
        fd[:,:3] = np.deg2rad(fd[:,:3]) 
        fd[:,:3] *= 50

        ## Compute framewise displacement (See Power 2012, 2014).
        fd = np.insert( np.abs( np.diff(fd, axis=0) ).sum(axis=1), 0, 0 )

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute motion regressors.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Remove trends.
        mc = detrend(mc, axis=0, type='constant')
        mc = detrend(mc, axis=0, type='linear')

        ## Perform PCA.
        pca = PCA(n_components=6)
        mc = pca.fit_transform(mc)

        ## Take only the number of components explaining 90% of the variance.
        varexp = np.cumsum(pca.explained_variance_ratio_)
        n_components = np.argmax(varexp >= 0.9) + 1
        mc = mc[:,:n_components]

        ## Save motion regressor.
        f = '%s/%s/msit_001/001/afMSIT.mc.par' %(mri_dir,subject)
        np.savetxt(f, mc, fmt='%s')
        info.write('%s,' %n_components)

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Write scrubbers.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        for threshold in thresholds:

            ## Find threshold violations.
            if not threshold: ix, = np.where(fd >= np.inf)
            else: ix, = np.where(fd >= threshold)

            ## Save.
            info.write('%s,' %len(ix))
            f = '%s/%s/msit_001/001/afMSIT.censor.%s.par' %(mri_dir,subject,threshold)
            if len(ix): np.savetxt(f, tr_onsets[ix,np.newaxis], fmt='%s')

        info.write('\n')

    info.close()
print 'Done.'

## Group Statistics

### Compute Beta Statistics

In [ ]:
import os
import numpy as np
import nibabel as nib
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for subject in subjects:
    
    print subject,
    
    for hemi in ['lh','rh']:
        
        ## Load data.
        f = os.path.join(mri_dir, subject, 'msit_001', '001', 'fmcpr.sm6.fsaverage.%s.b0dc.nii.gz' %hemi)
        obj = nib.load(f)
        
        ## Extract data and average over acquisitions.
        data = obj.get_data()
        data = np.apply_over_axes(np.mean, data, -1)
        
        ## Save.
        f = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi, 'betaconstant.nii.gz')
        obj = nib.Nifti1Image(data, obj.affine)
        nib.save(obj, f)
        
print 'Done.'

### Compute Percent Signal Change

In [ ]:
import os
import numpy as np
import nibabel as nib
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for subject in subjects:
    
    print subject,
    
    for hemi in ['lh','rh']:
        
        subj_dir = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi)
        
        ## Load constants.
        design_matrix = np.loadtxt(os.path.join(subj_dir, 'X.dat'))
        mean_signal = nib.load(os.path.join(subj_dir, 'betaconstant.nii.gz')).get_data()
        
        ## Iteratively compute PSC across conditions.
        for n, con in enumerate(['Neu','Int']):
            
            ## Load contrast.
            ces = nib.load(os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'ces.nii.gz'))
            affine = ces.affine
            ces = ces.get_data()
            
            ## Compute PSC.
            sf = design_matrix.max(axis=0)[n]
            psc = ces * sf / mean_signal * 100.
            
            ## Save.
            psc = nib.Nifti1Image(psc, affine)
            nib.save(psc, os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'psc.nii.gz'))
            
print 'Done.'

### Compute Group Contrasts

In [ ]:
import os
import numpy as np
import nibabel as nib
from statsmodels.api import WLS
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for hemi in ['lh','rh']:
    
    print 'Starting analysis of %s.' %hemi
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Loading subjects data.',
    
    psc, var = [], []
    for subject in subjects:
        
        ## Define subject directory.
        subj_dir = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi)
        
        for con in ['Neu','Int']:
            
            ## Load percent signal change.
            f = os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'psc.nii.gz')
            psc.append( nib.load(f).get_data() )
            
            ## Load contrast variance.
            f = os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'cesvar.nii.gz')
            obj = nib.load(f)
            var.append( obj.get_data() )
    
    ## Concatenate contrasts.
    psc = np.concatenate(psc, axis=-1).squeeze()
    var = np.concatenate(var, axis=-1).squeeze()
    affine = obj.affine
    print 'Finished.'
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively perform statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Compute contrast statistics.',
    
    ## Preallocate arrays.
    n_vert, n_cons = psc.shape
    Fvals, pvals = np.zeros((n_vert, 3)), np.zeros((n_vert, 3))

    ## Generate design matrix.
    X = np.zeros((n_cons,2))
    X[::2,0] = 1               # Neutral contrasts.
    X[1::2,1] = 1              # Interference contrasts.

    ## Generate weights.
    W = 1. / np.power(var, 2)

    ## Iterate over vertices.
    contrasts = [[1,0],[0,1],[-1,1]]
    for n in range(n_vert):

        ## Check if no data available.
        if np.any(np.isinf(W[n])): continue

        ## Fit model.
        model = WLS(psc[n], X, W[n]).fit()        
        
        ## Determine signs.
        signs = np.zeros(3)
        signs[:2] = np.sign(model.params)
        signs[-1] = np.sign(np.diff(model.params))
        
        ## Compute contrasts.
        for m, c in enumerate(contrasts):

            contrast = model.f_test(c)
            Fvals[n,m] = contrast.fvalue.max() * signs[m]
            pvals[n,m] = contrast.pvalue.max() * signs[m]

    ## Transform p-values.
    pvals = -np.log10(np.abs(pvals)) * np.sign(pvals)
    print 'Finished.'
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save results.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Saving results.',
    
    for n, contrast in enumerate(['neu','int','diff']):
        
        ## Make out directory.
        out_dir = 'fmri/afMSIT.%s.%s' %(contrast,hemi)
        if not os.path.isdir(out_dir): os.makedirs(out_dir)
        
        ## Save F-statistics.
        F = Fvals[:,n]
        for _ in range(2): F = F[:,np.newaxis]
        obj = nib.Nifti1Image(F, affine)
        nib.save(obj, os.path.join(out_dir,'F.mgz'))
        
        ## Save p-values.
        p = pvals[:,n]
        for _ in range(2): p = p[:,np.newaxis]
        obj = nib.Nifti1Image(p, affine)
        nib.save(obj, os.path.join(out_dir,'sig.mgz'))
            
    print 'Finished.'
    
print 'Done.'